<h1 style="text-align: center;">Tennis Analyzer ML Model V2</h1>
<h3 style="text-align: center;">Dan Warnick</h3>

<p>To start we will begin by selecting the data points we want to analyze with existing known results. For each data entry we will have two players each with the following data entries.</p>
<table style="font-size: .8em;">
    <tr>
        <th>Player Name</th>
    </tr>
</table>
<p>Along with match facts like Clay/Hard/Grass Court or Indoor/Outdoor. In the future may want to add weather and adjust certain parameters for more accuracy and more data points to train from, however for now this seems a good start.</p>

<h2>1.) Collect Data Efficiently</h2>

In [16]:
import django
from django.http import HttpResponse
from django.template import loader
import os
from django.db import models
import torch
from math import ceil, floor
import math
import numpy as np
from torch import nn
import torch.optim as optim
import torch.nn.functional as F
import torch.utils.data as data
from torch.utils.data import DataLoader, TensorDataset, WeightedRandomSampler
import torch.nn.utils as utils
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
import pandas as pd
from tqdm.notebook import tqdm
from datetime import datetime
from asgiref.sync import sync_to_async # type: ignore
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import roc_curve
import copy
import joblib

# os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'breakpoint.settings')
# django.setup()

# from render.models import *

In [17]:
start = '20130101'
end = '20231231'
match_type = 'm'

In [18]:
start_date = datetime.strptime(start, '%Y%m%d').date()
end_date = datetime.strptime(end, '%Y%m%d').date()

# if match_type == 'm':   
#     typer = MensTennisMatch
#     insert_db = MensTennisMatchStats
# else:
#     typer = WomensTennisMatch
#     insert_db = WomensTennisMatchStats

# query = insert_db.objects.filter(
#         tourney_date__range=(start_date, end_date)
#     ).order_by('tourney_date')
    
# games = await sync_to_async(list)(query.all().values())

df = pd.read_csv('../testcsvs/glickoUpdated.csv')
df['tourney_date'] = pd.to_datetime(df['tourney_date']).dt.date
df = df[(df['tourney_date'] >= start_date) & (df['tourney_date'] <= end_date)]

df = df.drop(['tourney_id', 'tourney_name', 'match_num', 'tourney_date', 'a_player_name', 'b_player_name', 'a_player_id', 'a_player_slug', 'b_player_id', 'b_player_slug', 'a_b_win', 'games', 'tiebreaks'], axis=1)

# df = pd.DataFrame(games).drop(['tourney_id', 'tourney_name', 'tourney_date', 'a_player_name', 'b_player_name', 'a_player_id', 'a_player_slug', 'b_player_id', 'b_player_slug','a_win_percent', 'a_serve_rating', 'a_return_rating', 'a_pressure_rating', 'a_avg_vs_elo', 'a_matches_played', 'b_win_percent', 'b_serve_rating', 'b_return_rating', 'b_pressure_rating', 'b_avg_vs_elo', 'b_matches_played', 'A_Odds', 'b_odds'], axis=1)

one_hot_encoder = OneHotEncoder(sparse_output=False)

# One-Hot Encode the 'category_text' column
category_encoded = one_hot_encoder.fit_transform(df[['surface']])

# Convert to DataFrame
category_encoded_df = pd.DataFrame(category_encoded, columns=one_hot_encoder.get_feature_names_out(['surface']))

# Concatenate the one-hot encoded columns back to the original DataFrame
df = pd.concat([df, category_encoded_df], axis=1)

# Drop the original 'category_text' column
df.drop('surface', axis=1, inplace=True)
print(len(df))

36813


In [19]:
df.head()

,best_of,tourney_round,a_player_age,a_player_rank,b_player_age,b_player_rank,glicko_rating_diff,a_glicko_rating,b_glicko_rating,a_glicko_rd,...,a_surface_return_second_won_glicko_rating,b_surface_second_won_glicko_rating,a_surface_return_second_won_glicko_rd,b_surface_second_won_glicko_rd,sets,a_odds,b_odds,surface_Clay,surface_Grass,surface_Hard
8138,3.0,0.8,25.0,63.0,28.0,73.0,109.440244,1748.578813,1639.138569,69.218597,...,1525.908593,1517.937658,66.528677,66.002918,0.0,1.91,1.83,0.0,0.0,1.0
8139,3.0,0.8,23.0,30.0,32.0,62.0,-2.699559,1753.795842,1756.495401,78.154224,...,1536.913003,1517.127728,70.116002,64.639126,1.0,1.65,2.18,0.0,0.0,1.0
8140,3.0,0.8,25.0,70.0,25.0,54.0,1.567183,1688.929747,1687.362564,86.599848,...,1491.133924,1508.334818,69.849716,65.220607,1.0,1.52,2.47,0.0,0.0,1.0
8141,3.0,0.8,28.0,376.0,25.0,104.0,-187.275163,1383.357011,1570.632174,292.237988,...,1471.098463,1515.147076,248.745451,70.303497,0.0,NaN,NaN,1.0,0.0,0.0
8142,3.0,0.8,22.0,26.0,27.0,59.0,223.167673,1932.516975,1709.349303,107.395435,...,1551.669156,1505.512790,84.977325,75.692922,1.0,1.63,2.22,1.0,0.0,0.0


In [20]:
# Create a scatter plot
# df = df[~(df == 1500).any(axis=1)]
# plt.figure(figsize=(5, 5))

# # Plot a_elo_rating vs b_elo_rating for a_b_win == 1
# x1 = df[df['a_b_win'] == 1]['a_recent_elo_rating']
# y1 = df[df['a_b_win'] == 1]['b_recent_elo_rating']
# plt.scatter(x1, y1, color='blue', label='Favorite Wins', s=.5, alpha=0.5)

# # Plot b_elo_rating vs a_elo_rating for a_b_win == 0
# x2 = df[df['a_b_win'] == 0]['b_recent_elo_rating']
# y2 = df[df['a_b_win'] == 0]['a_recent_elo_rating']
# plt.scatter(x2, y2, color='orange', label='Upset', s=.5, alpha=0.5)

# # Combine data for a single trendline
# combined_x = np.concatenate([x1, x2])
# combined_y = np.concatenate([y1, y2])

# # Fit a polynomial of degree 2 to the combined data
# print(combined_x)
# coefficients = np.polyfit(combined_x, combined_y, 3)
# polynomial = np.poly1d(coefficients)
# trendline_x = np.linspace(combined_x.min(), combined_x.max(), 100)
# trendline_y = polynomial(trendline_x)
# plt.plot(trendline_x, trendline_y, color='green', linewidth=1, label='Quadratic Trendline')

# # Setting the limits for x and y axis
# plt.xlim(1100, 2300)
# plt.ylim(1100, 2300)

# # Adding labels and title
# plt.xlabel('ELO Rating Winner')
# plt.ylabel('ELO Rating Loser')
# plt.title('Scatter Plot of ELO Ratings based on Win/Loss')
# plt.grid(True)
# plt.legend()

# # Show the plot
# plt.show()

In [21]:
# plt.figure(figsize=(8, 5))

# df['elo_diff'] = df['a_recent_elo_rating']

# # Create bins for ELO difference in intervals of 10
# bins = np.arange(df['elo_diff'].min(), df['elo_diff'].max() + 10, 10)
# labels = (bins[:-1] + bins[1:]) / 2
# df['elo_diff_bin'] = pd.cut(df['elo_diff'], bins=bins, labels=labels, include_lowest=True)

# # Calculate average win rate at each ELO difference bin
# average_win_rate = df.groupby('elo_diff_bin')['a_b_win'].mean().reset_index()
# average_win_rate.columns = ['elo_diff_bin', 'avg_win_rate']

# # Convert the bin labels to numeric values
# average_win_rate['elo_diff_bin'] = average_win_rate['elo_diff_bin'].astype(float)

# average_win_rate = average_win_rate[~np.isnan(average_win_rate).any(axis=1)]

# # Create the scatter plot
# plt.scatter(average_win_rate['elo_diff_bin'], average_win_rate['avg_win_rate'], color='blue', label='Average Win Rate', s=10, alpha=0.5)

# # Fit a polynomial of degree 3 to the average win rate data
# coefficients = np.polyfit(average_win_rate['elo_diff_bin'], average_win_rate['avg_win_rate'], 5)
# polynomial = np.poly1d(coefficients)
# trendline_x = np.linspace(-600, 600, 100)
# trendline_y = polynomial(trendline_x)
# plt.plot(trendline_x, trendline_y, color='green', linewidth=1, label='Cubic Trendline')

# # Setting the limits for x and y axis
# plt.xlim(-600, 600)
# plt.ylim(0, 1)

# # Adding labels and title
# plt.xlabel('ELO Rating Difference (Winner - Loser)')
# plt.ylabel('Average Win Rate')
# plt.title('Scatter Plot of ELO Rating Difference vs. Win Rate')
# plt.grid(True)
# plt.legend()

# # Show the plot
# plt.show()

In [22]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using mps device


In [23]:
class OutcomeProbability(nn.Module):
    def __init__(self, input_dim=36, dropout_prob=0.33):
        hidden_dim = int(input_dim * (2/3))
        super(OutcomeProbability, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.dropout1 = nn.Dropout(dropout_prob)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.dropout2 = nn.Dropout(dropout_prob)
        self.fc3 = nn.Linear(hidden_dim, 1)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout1(x)
        x = F.relu(self.fc2(x))
        x = self.dropout2(x)
        x = self.fc3(x)  # No sigmoid activation here
        return x

In [24]:
df = df.dropna()
df = df[~(df == .5).any(axis=1)]
df = df[~(df == -20).any(axis=1)]

# Extract odds and keep track of original indices
odds_df = df[['a_odds', 'b_odds']].copy()
odds_df['index'] = df.index

# Drop odds columns from the main DataFrame
df = df.drop(columns=['a_odds', 'b_odds'])

# Separate features and target variable
y = df['sets'].values
X = df.drop('sets', axis=1).values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test, idx_train, idx_test = train_test_split(
    X, y, odds_df['index'].values, test_size=1/5, random_state=42
)

# Scale the features
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

# Convert data to PyTorch tensors
train_dataset = data.TensorDataset(
    torch.tensor(X_train, dtype=torch.float32), 
    torch.tensor(y_train, dtype=torch.float32)
)
test_dataset = data.TensorDataset(
    torch.tensor(X_test, dtype=torch.float32), 
    torch.tensor(y_test, dtype=torch.float32)
)

# Create DataLoader objects
batch_size = 32
train_loader = data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Initialize the model, loss function, and optimizer
model = OutcomeProbability(input_dim=X.shape[1])
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Training loop
num_epochs = ceil(len(X_train) / batch_size)
for epoch in range(num_epochs):
    model.train()
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs).squeeze()
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    if (epoch + 1) % 25 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

# Testing loop
model.eval()
predictions = []
actuals = []
with torch.no_grad():
    test_loss = 0
    correct = 0
    total = 0
    for inputs, labels in test_loader:
        outputs = model(inputs).squeeze()
        loss = criterion(outputs, labels)
        test_loss += loss.item()
        predicted = (torch.sigmoid(outputs) > 0.5).float()
        predictions.extend(torch.sigmoid(outputs).numpy())
        actuals.extend(labels.numpy())
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'Test Loss: {test_loss / len(test_loader):.4f}, Accuracy: {accuracy:.2f}%')

# Convert predictions and actuals to numpy arrays
predictions = np.array(predictions)
actuals = np.array(actuals)

# Create a DataFrame with actuals and predictions
comparison_df = pd.DataFrame({
    'Actual': actuals,
    'Predicted': predictions
})
comparison_df['index'] = idx_test

# Merge the predictions with the original odds DataFrame
comparison_test_values = odds_df.set_index('index').loc[idx_test]
comparison_df = comparison_df.merge(
    comparison_test_values.reset_index(),
    on='index',
    suffixes=('_model', '_original')
)

KeyboardInterrupt: 

In [25]:
# Display the DataFrame
# results_df.head()
print(comparison_df)

      Actual  Predicted  index  a_odds  b_odds
0        1.0   0.557799   9710    5.34    1.15
1        0.0   0.449628  13827    1.11    6.35
2        1.0   0.030533  13354    1.34    3.18
3        0.0   0.590916  18891    1.76    2.03
4        0.0   0.241654  17974    6.51    1.11
...      ...        ...    ...     ...     ...
2714     0.0   0.383567  26876    2.16    1.70
2715     0.0   0.459142  22972    2.50    1.55
2716     1.0   0.293903  19462    4.86    1.18
2717     0.0   0.635385  14354    1.53    2.49
2718     1.0   0.417027  12956    2.47    1.53

[2719 rows x 5 columns]


In [26]:
# Kelly Criterion
def kelly_criterion(vegas_odds, calculated_probability):
    vegas_odds -= 1
    # Calculate the Kelly fraction
    kelly_fraction = calculated_probability - ((1 - calculated_probability)/vegas_odds)
    # print(kelly_fraction)
    
    # Ensure that the fraction is not negative
    kelly_fraction = max(0, kelly_fraction)
    
    return kelly_fraction

In [31]:
better = 0
total_won = 0
diff_fav = 0
bet_correct = 0

model_correct = 0

wrong = 0
comparison_df = comparison_df.dropna()
length = len(comparison_df)

confidence_pct = .5
confidence_top_pct = 1

avg_bet = 0

OVER = 2.56
UNDER = 1.48

UNIT = 30

for i, row in comparison_df.iterrows():

    if confidence_top_pct > row['Predicted'] > confidence_pct :
        kelly = (kelly_criterion(OVER, row['Predicted']) * UNIT)
        better += 1 if kelly > 0 else 0
        avg_bet += kelly
        if(row['Actual'] == 1):
            bet_correct += 1
            total_won += (OVER-1) * kelly
        else:
            wrong += 1
            total_won -= kelly

    # if 1-confidence_top_pct < row['Predicted'] < 1-confidence_pct :
    #     kelly = (kelly_criterion(UNDER, 1-row['Predicted']) * UNIT)
    #     better += 1 if kelly > 0 else 0
    #     avg_bet += kelly
    #     if(row['Actual'] == 0):
    #         bet_correct += 1
    #         total_won += (UNDER-1) * kelly
    #     else:
    #         wrong += 1
    #         total_won -= kelly


    if round(row['Predicted']) == round(row['Actual']):
        model_correct += 1


print(f"Total won on ${UNIT} bankroll bets: {total_won:.2f} on a total # bets: {better} from a total of {length} games")
print(f"Avg_bet_size ${avg_bet/better}")
print(f"Amount of incorrect bets : {wrong/better}")
print(f"Correct Bets: {bet_correct/better}")
print(f"Model % Correct : {model_correct/length}")


Total won on $30 bankroll bets: 94.56 on a total # bets: 292 from a total of 2719 games
Avg_bet_size $9.800283192834556
Amount of incorrect bets : 0.6164383561643836
Correct Bets: 0.3835616438356164
Model % Correct : 0.6057374034571533
